In [ ]:
!pip install orb-models
!pip install --extra-index-url=https://pypi.nvidia.com "cuml-cu12==25.2.*"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 129.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: dm-tree
    Found existing installation: dm-tree 0.1.9
    Uninstalling dm-tree-0.1.9:
      Successfully uninstalled dm-tree-0.1.9
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 201.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 145.

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-


import os, csv, gc, io, re
import torch
from ase.io import read, write
from ase.constraints import FixAtoms
from ase.optimize import BFGS, QuasiNewton
from ase.mep.neb import NEB
from orb_models.forcefield import pretrained
from orb_models.forcefield.calculator import ORBCalculator

# ───────────────────────────────────────────────────────────────────────────────
# Google‑Drive mount (comment if running locally)
# ───────────────────────────────────────────────────────────────────────────────
from google.colab import drive
drive.mount('/content/drive')


def read_poscar_no_junk(path: str):
    """Read POSCAR/CONTCAR."""
    try:
        return read(path)
    except AssertionError:
        pass

    with open(path) as fh:
        lines = fh.readlines()

    num_line = next(i for i, ln in enumerate(lines)
                    if re.fullmatch(r'\s*\d+(?:\s+\d+)*\s*', ln))
    natoms   = sum(map(int, lines[num_line].split()))
    coord_hdr = next(i for i in range(num_line + 1, len(lines))
                     if lines[i].strip().lower().startswith(('direct', 'cart')))
    last = coord_hdr + 1 + natoms
    return read(io.StringIO(''.join(lines[:last])), format='vasp')

# ───────────────────────────────────────────────────────────────────────────────
# ───────────────────────────────────────────────────────────────────────────────
device = "cuda" if torch.cuda.is_available() else "cpu"
def make_calc():
    return ORBCalculator(pretrained.orb_v3_conservative_inf_omat(
  device=device,
  precision="float32-high",))

# ───────────────────────────────────────────────────────────────────────────────
# 3. Parameters
# ───────────────────────────────────────────────────────────────────────────────
BASE_DIR        = "/content/drive/MyDrive/MLIP-NEB-inputs"
SKIP_SYSTEM     = "#"
IMAGES_BETWEEN  = 7
FMAX            = 0.05
MAX_STEPS       = 1000

# ───────────────────────────────────────────────────────────────────────────────
# 4. Main loop
# ───────────────────────────────────────────────────────────────────────────────
for root, dirs, _ in os.walk(BASE_DIR):
    if not ({"initial", "final"} <= set(dirs)):
        continue

    system = os.path.basename(root)
    if system == SKIP_SYSTEM:
        print(f"{system} (user skip)")
        continue

    save_dir = os.path.join(root, "ORBV3_run")
    csv_path = os.path.join(save_dir, "energies.csv")
    if os.path.exists(csv_path):
        print(f"{system} (already done)")
        continue

    print(f"\n Processing {system}")
    os.makedirs(save_dir, exist_ok=True)

    # ── read endpoints ────────────────────────────────────────────────────
    initial = read_poscar_no_junk(os.path.join(root, "initial", "POSCAR"))
    final   = read_poscar_no_junk(os.path.join(root, "final",   "POSCAR"))

    # freeze framework atoms (tag > 1) and attach calculators
    for atoms in (initial, final):
        atoms.set_constraint(FixAtoms([a.tag > 1 for a in atoms]))
        atoms.calc = make_calc()

    # ── relax endpoints ───────────────────────────────────────────────────
    for atoms, lbl in ((initial, "initial"), (final, "final")):
        traj = os.path.join(save_dir, f"{lbl}_relaxed.traj")
        BFGS(atoms, trajectory=traj).run(fmax=FMAX, steps=MAX_STEPS)
        write(os.path.join(save_dir, f"relaxed_{lbl}.vasp"), atoms, "vasp")

    # reload relaxed structures
    initial = read(os.path.join(save_dir, "initial_relaxed.traj"))
    final   = read(os.path.join(save_dir, "final_relaxed.traj"))
    fixed   = FixAtoms([a.tag > 1 for a in final])

    # ── build NEB band ─────────────────
    images = [initial]
    for _ in range(IMAGES_BETWEEN):
        img = initial.copy()
        img.set_constraint(fixed)
        img.calc = make_calc()
        images.append(img)
    images.append(final)

    # final already has a calc, ensure it's independent
    final.calc = make_calc()

    # NEB 
    neb = NEB(images, parallel=True, k=5, method="eb")
    neb.interpolate("idpp")
    BFGS(neb, trajectory=os.path.join(save_dir, "neb.traj")).run(
        fmax=FMAX, steps=MAX_STEPS
    )

    # ── energies & migration barrier ─────────────────────────────────────
    energies = []
    with open(csv_path, "w", newline="") as fh:
        wr = csv.writer(fh); wr.writerow(["Image", "Energy (eV)"])
        for i, img in enumerate(images):
            try:
                e = img.get_potential_energy()
            except Exception as err:
                print(f"⚠️  {system} img{i}: {err}")
                e = None
            energies.append(e)
            write(os.path.join(save_dir, f"neb_bfgs_{i}.vasp"), img, "vasp")
            wr.writerow([i, e])

    if None not in energies:
        barrier = max(energies) - min(energies)
        with open(csv_path, "a", newline="") as fh:
            csv.writer(fh).writerow([])
            csv.writer(fh).writerow(["Migration Barrier", barrier])
        print(f"{system}: barrier = {barrier:.4f} eV")
    else:
        print(f"{system}: barrier skipped (missing energies)")

    # tidy GPU memory
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

print("\n All NEB calculations complete!")

Mounted at /content/drive
⏭️  Mn2S3O12 (already done)
⏭️  CaMn2S3O12 (already done)
⏭️  Ca2.5V2P3O12 (already done)
⏭️  Ca0.5V2P3O12 (already done)
⏭️  Ca2.5Mn2P3O12 (already done)
⏭️  Ca0.5Mn2P3O12 (already done)

🚀  Processing Fe2S3O12


/usr/local/lib/python3.11/dist-packages/orb_models/utils.py:30: UserWarning: Setting global torch default dtype to torch.float32.
  warnings.warn(f"Setting global torch default dtype to {torch_dtype}.")


Output()

W0707 10:39:28.852000 1619 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode


                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 10:39:51     -221.802994       1.0181
BFGSLineSearch:    1[  3] 10:39:51     -221.954239       0.7127
BFGSLineSearch:    2[  5] 10:39:51     -222.015045       1.1550
BFGSLineSearch:    3[  7] 10:39:51     -222.116364       1.4419
BFGSLineSearch:    4[  9] 10:39:52     -222.176239       0.6748
BFGSLineSearch:    5[ 11] 10:39:52     -222.231781       0.3620
BFGSLineSearch:    6[ 13] 10:39:52     -222.239960       0.1418
BFGSLineSearch:    7[ 15] 10:39:52     -222.246506       0.2759
BFGSLineSearch:    8[ 17] 10:39:52     -222.251831       0.1007
BFGSLineSearch:    9[ 19] 10:39:52     -222.253830       0.1334
BFGSLineSearch:   10[ 21] 10:39:52     -222.258774       0.1048
BFGSLineSearch:   11[ 22] 10:39:52     -222.263580       0.2218
BFGSLineSearch:   12[ 32] 10:39:52     -222.264771       0.1591
BFGSLineSearch:   13[ 34] 10:39:52     -222.266403       0.2119
BFGSLineSearch:   14[ 36] 10:39:52     

/usr/local/lib/python3.11/dist-packages/orb_models/utils.py:30: UserWarning: Setting global torch default dtype to torch.float32.
  warnings.warn(f"Setting global torch default dtype to {torch_dtype}.")


      Step     Time          Energy          fmax
BFGS:    0 10:41:55     -219.091934        6.224322
BFGS:    1 10:41:55     -219.826172        3.220141
BFGS:    2 10:41:55     -220.162979        1.742705
BFGS:    3 10:41:56     -220.322418        0.963779
BFGS:    4 10:41:56     -220.412003        0.924340
BFGS:    5 10:41:56     -220.509903        0.918942
BFGS:    6 10:41:56     -220.578705        0.874170
BFGS:    7 10:41:57     -220.625900        0.664304
BFGS:    8 10:41:57     -220.658295        0.542973
BFGS:    9 10:41:57     -220.698669        0.499076
BFGS:   10 10:41:57     -220.734909        0.523300
BFGS:   11 10:41:58     -220.761093        0.628560
BFGS:   12 10:41:58     -220.775757        2.048780
BFGS:   13 10:41:58     -220.796402        0.513622
BFGS:   14 10:41:59     -220.804855        0.327653
BFGS:   15 10:41:59     -220.834686        0.242393
BFGS:   16 10:41:59     -220.842056        0.249628
BFGS:   17 10:41:59     -220.859085        0.413843
BFGS:   18 10: